# 텍스트 분석2-7 문서 군집화

### 2. Opinion Review 데이터 세트를 이용한 문서 군집화 수행

In [1]:
import pandas as pd
import glob, os

path=r'C:\Users\각자의 디렉토리 설정\OpinosisDataset1.0\topics'

# path로 지정한 디렉터리 밑에 있는 모든 .data 파일의 파일명을 리스트로 취합
all_files=glob.glob(os.path.join(path, "*.data"))
filename_list=[]
opinion_text=[]

In [14]:
# 개별 파일을 하나씩 읽어서 파일명과 파일리뷰를 하나의 DataFrame으로 로드

for file_ in all_files:
    # 개별 파일을 dataframe으로 생성
    df=pd.read_table(file_,index_col=None, header=0, encoding='latin1')
    
    #절대 경로로 주어진 파일명에서 파일명.txt만 추출
    filename_=file_.split('\\')[-1] 
    filename=filename_.split('.')[0]
    
    filename_list.append(filename)
    opinion_text.append(df.to_string())
    
    
document_df=pd.DataFrame({'filename':filename_list, 'opinion_text':opinion_text})
document_df.head()

C:\anaconda3\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  """


,filename,opinion_text
0,accuracy_garmin_nuvi_255W_gps,", and is very, very acc..."
1,bathroom_bestwestern_hotel_sfo,"The room was not overly big, but clean and..."
2,battery-life_amazon_kindle,After I plugged it in to my USB hub on my ...
3,battery-life_ipod_nano_8gb,short battery life I moved up from a...
4,battery-life_netbook_1005ha,"6GHz 533FSB cpu, glossy display, 3, Cell 2..."


#### 문서를 TF-IDF 형태로 피처 벡터화

In [31]:
from nltk.stem import WordNetLemmatizer
import nltk
import string

remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)
lemmar = WordNetLemmatizer()

# 단어의 원형을 찾음
def LemTokens(tokens):
    return [lemmar.lemmatize(token) for token in tokens]

# 주어진 텍스트를 소문자로 만들고 remove_punct_dict에 대한 참조를 사용하여 텍스트 내 문장 부호를 제거
# 이후 단어토큰화를 수행하고 LemToken()을 수행
def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

#### K-Means를 통해 군집화 수행

In [23]:
from sklearn.feature_extraction.text import TfidfVectorizer

# tokenizer는 Lemmatization을 구현한 LemNormalize()함수를 이용
tfidf_vect=TfidfVectorizer(tokenizer=LemNormalize, stop_words='english',
                          ngram_range=(1,2), min_df=0.05, max_df=0.85)

# opinion_text 칼럼 값으로 피처벡터화 수행
feature_vect=tfidf_vect.fit_transform(document_df['opinion_text'])

C:\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:301: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [41]:
from sklearn.cluster import KMeans

# 3개의 집합으로 군집화 
km_cluster = KMeans(n_clusters=3, max_iter=10000, random_state=0)
km_cluster.fit(feature_vect)
cluster_label = km_cluster.labels_

document_df['cluster_label'] = cluster_label
document_df.head()

,filename,opinion_text,cluster_label
0,accuracy_garmin_nuvi_255W_gps,", and is very, very acc...",1
1,bathroom_bestwestern_hotel_sfo,"The room was not overly big, but clean and...",2
2,battery-life_amazon_kindle,After I plugged it in to my USB hub on my ...,1
3,battery-life_ipod_nano_8gb,short battery life I moved up from a...,1
4,battery-life_netbook_1005ha,"6GHz 533FSB cpu, glossy display, 3, Cell 2...",1


In [59]:
# 자동차 리뷰
document_df[document_df['cluster_label']==0].sort_values(by='filename').head()

,filename,opinion_text,cluster_label
6,comfort_honda_accord_2008,"Drivers seat not comfortable, the car its...",0
57,comfort_honda_accord_2008,"Drivers seat not comfortable, the car its...",0
7,comfort_toyota_camry_2007,Ride seems comfortable and gas mileage fa...,0
58,comfort_toyota_camry_2007,Ride seems comfortable and gas mileage fa...,0
16,gas_mileage_toyota_camry_2007,Ride seems comfortable and gas mileage fa...,0


In [43]:
# 전자기기 리뷰
document_df[document_df['cluster_label']==1].sort_values(by='filename').head()

,filename,opinion_text,cluster_label
0,accuracy_garmin_nuvi_255W_gps,", and is very, very acc...",1
51,accuracy_garmin_nuvi_255W_gps,", and is very, very acc...",1
53,battery-life_amazon_kindle,After I plugged it in to my USB hub on my ...,1
2,battery-life_amazon_kindle,After I plugged it in to my USB hub on my ...,1
3,battery-life_ipod_nano_8gb,short battery life I moved up from a...,1


In [44]:
# 호텔 리뷰
document_df[document_df['cluster_label']==2].sort_values(by='filename').head()

,filename,opinion_text,cluster_label
1,bathroom_bestwestern_hotel_sfo,"The room was not overly big, but clean and...",2
52,bathroom_bestwestern_hotel_sfo,"The room was not overly big, but clean and...",2
13,food_holiday_inn_london,The room was packed to capacity with queu...,2
64,food_holiday_inn_london,The room was packed to capacity with queu...,2
14,food_swissotel_chicago,The food for our event was deli...,2


## 3. 군집별 핵심단어 추출하기

In [47]:
cluster_centers = km_cluster.cluster_centers_
print('cluster_centers shape: ', cluster_centers.shape)
print(cluster_centers)

cluster_centers shape:  (3, 2409)
[[0.         0.00155233 0.         ... 0.         0.         0.        ]
 [0.0174125  0.         0.         ... 0.         0.         0.00462558]
 [0.         0.00174184 0.0026023  ... 0.00331835 0.00357495 0.        ]]


In [52]:
# 군집별 상위 10개의 핵심단어, 그 단어의 중심 위치 상대값, 대상 파일명들을 반환함. 

def get_cluster_details(cluster_model, cluster_data, feature_names, clusters_num, top_n_features=10):    
    cluster_details = {}  # 핵심단어 이름과 상대위치 값을 추출해 저장하는 딕셔너리
    
    # cluster_centers array 의 값이 큰 순으로 정렬된 index 값을 반환
    # 군집 중심점(centroid)별 할당된 word 피처들의 거리값이 큰 순으로 값을 구하기 위함.  
    centroid_feature_ordered_ind = cluster_model.cluster_centers_.argsort()[:,::-1]
    
    #개별 군집별로 iteration하면서 핵심단어, 그 단어의 중심 위치 상대값, 대상 파일명 입력
    for cluster_num in range(clusters_num):
        # 개별 군집별 정보를 담을 데이터 초기화. 
        cluster_details[cluster_num] = {}
        cluster_details[cluster_num]['cluster'] = cluster_num
        
        # cluster_centers_.argsort()[:,::-1] 로 구한 index 를 이용하여 top n 피처 단어를 구함. 
        top_feature_indexes = centroid_feature_ordered_ind[cluster_num, :top_n_features]
        top_features = [ feature_names[ind] for ind in top_feature_indexes ]
        
        # top_feature_indexes를 이용해 해당 피처 단어의 중심 위치 상댓값 구함 
        top_feature_values = cluster_model.cluster_centers_[cluster_num, top_feature_indexes].tolist()
        
        # cluster_details 딕셔너리 객체에 개별 군집별 핵심 단어와 중심위치 상대값, 해당 파일명 입력
        cluster_details[cluster_num]['top_features'] = top_features
        cluster_details[cluster_num]['top_features_value'] = top_feature_values
        filenames = cluster_data[cluster_data['cluster_label'] == cluster_num]['filename']
        filenames = filenames.values.tolist()
        cluster_details[cluster_num]['filenames'] = filenames
        
    return cluster_details

In [56]:
feature_names = tfidf_vect.get_feature_names()

cluster_details = get_cluster_details(cluster_model=km_cluster, cluster_data=document_df,
                                  feature_names=feature_names, clusters_num=3, top_n_features=10 )


# 각 cluster의 상위 feature와 파일명 출력
for cluster_num, cluster_detail in cluster_details.items():
    print('####### Cluster {0}'.format(cluster_num))
    print('Top features:', cluster_detail['top_features'])
    print('Reviews 파일명 :',cluster_detail['filenames'][:7])
    print('==================================================')
    
# cluster0 은 자동차 리뷰 군집으로, 실내 인테리어, 좌석, 연료 효율 등이 핵심단어로 군집화 되었음
# cluster1 은 전자제품 리뷰 군집로 화면과 배터리 수명 등이 핵심단어로 군집화 되었음
# cluster2 는 호텔 리뷰 군집으로 룸과 서비스 등이 핵심 단어로 군집화되었다.

####### Cluster 0
Top features: ['interior', 'seat', 'mileage', 'comfortable', 'car', 'gas', 'gas mileage', 'comfort', 'ride', 'performance']
Reviews 파일명 : ['comfort_honda_accord_2008', 'comfort_toyota_camry_2007', 'gas_mileage_toyota_camry_2007', 'interior_honda_accord_2008', 'interior_toyota_camry_2007', 'mileage_honda_accord_2008', 'performance_honda_accord_2008']
####### Cluster 1
Top features: ['screen', 'battery', 'life', 'battery life', 'keyboard', 'kindle', 'size', 'button', 'voice', 'easy']
Reviews 파일명 : ['accuracy_garmin_nuvi_255W_gps', 'battery-life_amazon_kindle', 'battery-life_ipod_nano_8gb', 'battery-life_netbook_1005ha', 'buttons_amazon_kindle', 'directions_garmin_nuvi_255W_gps', 'display_garmin_nuvi_255W_gps']
####### Cluster 2
Top features: ['room', 'hotel', 'service', 'location', 'staff', 'food', 'clean', 'bathroom', 'parking', 'room wa']
Reviews 파일명 : ['bathroom_bestwestern_hotel_sfo', 'food_holiday_inn_london', 'food_swissotel_chicago', 'free_bestwestern_hotel_sfo',